In [ ]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration

from data import DataCollatorSpeechSeq2SeqWithPadding, WhisperAudioCaptionProcessor

In [ ]:
name = "openai/whisper-tiny"
ckpt_dir = "/home/seastar105/Work/audio-caption/logs/tiny-test/runs/2024-12-25_15-32-56/checkpoint-10000"

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained(ckpt_dir)

In [ ]:
processor = WhisperAudioCaptionProcessor(name)
tokenizer = processor.tokenizer
collator = DataCollatorSpeechSeq2SeqWithPadding(processor)

In [ ]:
dataset = (
    load_dataset("krishnakalyan3/emo_webds_2", split="train", streaming=True)
    .map(processor, batched=False)
    .select_columns(["input_features", "labels"])
)

In [ ]:
batch_size = 4
iterator = iter(dataset)
batch = [next(iterator) for _ in range(batch_size)]

In [ ]:
inputs = collator(batch)
input_features = inputs.input_features
for caption in tokenizer.batch_decode(inputs.labels, skip_special_tokens=True):
    print(caption)

In [ ]:
outputs = model.generate(**inputs, use_cache=True, do_sample=True, temperature=1.0)

In [ ]:
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for sample in result:
    print(sample)